<a href="https://colab.research.google.com/github/LazaroR-u/Final_project_QxQ_23-24/blob/main/Final_project_QubitxQubit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
%%capture
pip install cirq

In [106]:
import numpy as np
import cirq
#from cirq.contrib.svg import SVGCircuit
#import matplotlib.pyplot as plt

## QUANTUM BB84 PROTOCOL

Example program to demonstrate BB84 QKD Protocol

BB84 [1] is a quantum key distribution (QKD) protocol developed by
Charles Bennett and Gilles Brassard in 1984. It was the first quantum
cryptographic protocol, using the laws of quantum mechanics (specifically,
no-cloning) to provide provably secure key generation.

BB84 relies on the fact that it is impossible to gain information
distinguishing two non-orthogonal states without disturbing the signal.

The scheme involves two parties Alice and Bob connected by a classical
communication channel. In addition to this, Alice can also prepare
qubits in a particular state and send them to Bob using a unidirectional
quantum channel.

Alice generates two random binary strings a and b of the same length n.
The string a encodes the state and the string b encodes the basis.
She then prepares n qubits according to the following prescription:

|q[i]⟩ = |0⟩ if a[i] == 0 and b[i] == 0

|q[i]⟩ = |1⟩ if a[i] == 1 and b[i] == 0

|q[i]⟩ = |+⟩ if a[i] == 0 and b[i] == 1

|q[i]⟩ = |-⟩ if a[i] == 1 and b[i] == 1


where |+/-⟩ = 1/sqrt(2)*(|0⟩+/-|1⟩).

Alice sends her qubits to Bob. Bob then generates a random binary string
c of length n. He measures the qubit |q[i]⟩ in the {|0⟩, |1⟩} basis
(computational basis) if c[i] == 0 and in the {|+⟩,|-⟩} basis
(Hadamard basis) if c[i] == 1 and stores the result in a string m.
Alice and Bob then announce the strings b and c, which encode
the random basis choices of Alice and Bob respectively.

The strings a and m match in the places where b and c are the same.
This happens because the state was measured in the same basis in
which it was prepared. For the remaining bits, the results are
uncorrelated. The bits from strings a and m where the bases match
can be used as a key for cryptography.

BB84 is secure against intercept-and-resend attacks. The no-cloning
theorem [2] guarantees that a qubit that is in an unknown state to
begin with cannot be copied or cloned. Thus, any measurement will
destroy the initial state of the qubit. Suppose an eavesdropper Eve
intercepts all of Alice's qubits, measures them in a randomly chosen
basis, prepares another qubit in the state that she measured and resends
it to Bob. The state Eve measures is not necessarily the state Alice
prepared,  and hence, Alice and Bob will not measure the same outcome
for that qubit even if their basis choices match. Thus, Alice and Bob
can detect eavesdropping by comparing a few bits from their
obtained keys.

[1]: https://en.wikipedia.org/wiki/BB84


[2]: https://en.wikipedia.org/wiki/No-cloning_theorem

 === Example output ===

Simulating non-eavesdropped protocol

0: ───X───M───────────

1: ───H───H───M───────

2: ───X───H───M───────

3: ───X───H───M───────

4: ───X───H───M───────

5: ───X───H───H───M───

6: ───H───M───────────

7: ───H───H───M───────

Alice's basis:  CHCCCHCH

Bob's basis:    CHHHHHHH

Alice's bits:   10111100

Bases match::   XX___X_X

Expected key:   1010

Actual key:     1010

Simulating eavesdropped protocol

0: ───H───M───────────H───M───────────

1: ───H───M───────────H───H───M───────

2: ───X───H───H───M───X───H───H───M───

3: ───H───M───────────H───M───────────

4: ───M───────────────M───────────────

5: ───X───H───M───────X───H───M───────

6: ───H───M───────────X───H───M───────

7: ───X───H───H───M───X───H───M───────

Alice's basis:  HCHCCHCH

Bob's basis:    HHHCCHCC

Alice's bits:   00100101

Bases match::   X_XXXXX_

Expected key:   010010

Actual key:     111011


In [107]:
def main(num_qubits=28):
    # Setup non-eavesdropped protocol
    print('Simulating non-eavesdropped protocol')
    qubits = cirq.LineQubit.range(num_qubits)
    alice_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]
    alice_state = [np.random.randint(0, 2) for _ in range(num_qubits)]
    bob_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]

    expected_key = bitstring(
        [alice_state[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
    )

    circuit = make_bb84_circ(num_qubits, alice_basis, bob_basis, alice_state)

    # Run simulations.
    repetitions = 1

    result = cirq.Simulator().run(program=circuit, repetitions=repetitions)
    result_bitstring = bitstring([int(np.ravel(result.measurements[str(q)])[0]) for q in qubits])

    # Take only qubits where bases match
    obtained_key = ''.join(
        [result_bitstring[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
    )

    assert expected_key == obtained_key, "Keys don't match"

    print(circuit)
    print_results(alice_basis, bob_basis, alice_state, expected_key, obtained_key)

    if expected_key == obtained_key:
      print(f"There is not eavesdropped. Take the secure key: {obtained_key}")
    else:
      print("There is eavesdropped. The key is not secure. Try again!")
      print(f"expected key: {expected_key}, obtained key: {obtained_key}")

    # Setup eavesdropped protocol
    print("-----------------------------------------------------------")
    print('Simulating eavesdropped protocol')
    np.random.seed(200)  # Seed random generator for consistent results
    alice_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]
    alice_state = [np.random.randint(0, 2) for _ in range(num_qubits)]
    bob_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]
    eve_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]

    expected_key = bitstring(
        [alice_state[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
    )

    # Eve intercepts the qubits

    alice_eve_circuit = make_bb84_circ(num_qubits, alice_basis, eve_basis, alice_state)

    # Run simulations.
    repetitions = 1
    result = cirq.Simulator().run(program=alice_eve_circuit, repetitions=repetitions)
    eve_state = [int(np.ravel(result.measurements[str(q)])[0]) for q in qubits]

    eve_bob_circuit = make_bb84_circ(num_qubits, eve_basis, bob_basis, eve_state)

    # Run simulations.
    repetitions = 1
    result = cirq.Simulator().run(program=eve_bob_circuit, repetitions=repetitions)
    result_bitstring = bitstring([int(np.ravel(result.measurements[str(q)])[0]) for q in qubits])


    # Take only qubits where bases match
    obtained_key = ''.join(
        [result_bitstring[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
    )

    assert expected_key != obtained_key, "Keys shouldn't match"

    circuit = alice_eve_circuit + eve_bob_circuit

    print(circuit)
    print_results(alice_basis, bob_basis, alice_state, expected_key, obtained_key)

    if expected_key == obtained_key:
      print(f"There is not eavesdropped. Take the secure key: {obtained_key}")
    else:
      print("There is eavesdropped. The key is not secure. Try again!")
      print(f"expected key: {expected_key}, obtained key: {obtained_key}")



def make_bb84_circ(num_qubits, alice_basis, bob_basis, alice_state):

    qubits = cirq.LineQubit.range(num_qubits)

    circuit = cirq.Circuit()

    # Alice prepares her qubits
    alice_enc = []
    for index, _ in enumerate(alice_basis):
        if alice_state[index] == 1:
            alice_enc.append(cirq.X(qubits[index]))
        if alice_basis[index] == 1:
            alice_enc.append(cirq.H(qubits[index]))

    circuit.append(alice_enc)

    # Bob measures the received qubits
    bob_basis_choice = []
    for index, _ in enumerate(bob_basis):
        if bob_basis[index] == 1:
            bob_basis_choice.append(cirq.H(qubits[index]))

    circuit.append(bob_basis_choice)
    circuit.append(cirq.measure_each(*qubits))

    return circuit


def bitstring(bits):
    return ''.join(str(int(b)) for b in bits)

def int_to_bin(n: int, zeros: int ):
    """
    Transform an integer to binary format.
    Parameters:
    n (int): The number of bits to generate.
    zeros (int): number of qubits.
    Returns:
    np.ndarray: An array of random bits.
    """
    return bin(n)[2:].zfill(zeros)

def print_results(alice_basis, bob_basis, alice_state, expected_key, obtained_key):
    num_qubits = len(alice_basis)
    basis_match = ''.join(
        ['X' if alice_basis[i] == bob_basis[i] else '_' for i in range(num_qubits)]
    )
    alice_basis_str = "".join(['C' if alice_basis[i] == 0 else "H" for i in range(num_qubits)])
    bob_basis_str = "".join(['C' if bob_basis[i] == 0 else "H" for i in range(num_qubits)])

    print(f'Alice\'s basis:\t{alice_basis_str}')
    print(f'Bob\'s basis:\t{bob_basis_str}')
    print(f'Alice\'s bits:\t{bitstring(alice_state)}')
    print(f'Bases match::\t{basis_match}')
    print(f'Expected key:\t{expected_key}')
    print(f'Actual key:\t{obtained_key}')
    print(f"Keys are the same:\t{expected_key == obtained_key}")


if __name__ == "__main__":
    main()

Simulating non-eavesdropped protocol
0: ────X───H───M───────

1: ────X───H───H───M───

2: ────H───H───M───────

3: ────X───H───M───────

4: ────H───H───M───────

5: ────X───H───M───────

6: ────X───M───────────

7: ────X───H───M───────

8: ────M───────────────

9: ────M───────────────

10: ───H───M───────────

11: ───M───────────────

12: ───X───H───M───────

13: ───H───M───────────

14: ───X───M───────────

15: ───X───H───M───────

16: ───X───H───M───────

17: ───X───M───────────

18: ───X───M───────────

19: ───H───M───────────

20: ───H───H───M───────

21: ───H───M───────────

22: ───X───M───────────

23: ───M───────────────

24: ───H───M───────────

25: ───X───H───M───────

26: ───X───H───H───M───

27: ───X───H───H───M───
Alice's basis:	HHHHHCCCCCCCCHCCCCCCHCCCCCHH
Bob's basis:	CHHCHHCHCCHCHCCHHCCHHHCCHHHH
Alice's bits:	1101011100001011111000100111
Bases match::	_XX_X_X_XX_X__X__XX_X_XX__XX
Expected key:	100100011101011
Actual key:	100100011101011
Keys are the same:	True
There is n

## CREATE KEY

In [108]:
def create_key_bb84(num_qubits=30, eavesdropper=False):
    qubits = cirq.LineQubit.range(num_qubits)

    if not eavesdropper:
        alice_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]
        alice_state = [np.random.randint(0, 2) for _ in range(num_qubits)]
        bob_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]

        expected_key = ''.join(
            [str(alice_state[i]) for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
        )

        circuit = make_bb84_circ(num_qubits, alice_basis, bob_basis, alice_state)

        result = cirq.Simulator().run(program=circuit, repetitions=1)
        result_bitstring = bitstring([int(np.ravel(result.measurements[str(q)])[0]) for q in qubits])

        obtained_key = ''.join(
            [result_bitstring[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
        )

        if expected_key == obtained_key:
            return expected_key
        else:
            print("There is eavesdropped. The key is not secure. Try again!")
            print(f"expected key: {expected_key}, obtained key: {obtained_key}")

    elif eavesdropper:
        alice_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]
        alice_state = [np.random.randint(0, 2) for _ in range(num_qubits)]
        bob_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]
        eve_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]

        expected_key = bitstring(
            [alice_state[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
        )

        # Eve intercepts the qubits
        alice_eve_circuit = make_bb84_circ(num_qubits, alice_basis, eve_basis, alice_state)

        # Run simulations.
        repetitions = 1
        result = cirq.Simulator().run(program=alice_eve_circuit, repetitions=repetitions)
        eve_state = [int(np.ravel(result.measurements[str(q)])[0]) for q in qubits]

        eve_bob_circuit = make_bb84_circ(num_qubits, eve_basis, bob_basis, eve_state)

        # Run simulations.
        repetitions = 1
        result = cirq.Simulator().run(program=eve_bob_circuit, repetitions=repetitions)
        result_bitstring = bitstring([int(np.ravel(result.measurements[str(q)])[0]) for q in qubits])

        # Take only qubits where bases match
        obtained_key = ''.join(
            [result_bitstring[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
        )

        if expected_key != obtained_key:
          print("There is eavesdropped. The key is not secure. Try again!")
          print(f"expected key: {expected_key}, obtained key: {obtained_key}")

        else:
          print(f"There is not eavesdropped. Take the secure 999key: {obtained_key}")

create_key_bb84(30, eavesdropper=False)

'01011000000001111'

## ENCRYPT - DECRYPT - TRANSFORM FUNCTIONS

In [109]:
%%capture
pip install endecrypt

In [110]:
import endecrypt

In [111]:
def shorten_key(data, key):
    # Shortens key if it is longer than the given data
    # The key should be the same length as the data for the binary addition
    if len(key) < len(data):
        repeat_factor = (len(data) // len(key)) + 1
        new_key = (key * repeat_factor)[:len(data)]
        return new_key
    else:
        return key


def encryption(data,key):
    # function which creates the encrypted message using the key created between Alice and Bob
    # using simple binary addition
    # Used by Alice

    if len(data) != len(key):
        # if the key is longer than the data, shorten the key
        key = shorten_key(data,key)

    encrypted_message = [None for j in range(len(data))]

    # binary addition
    for i in range(len(data)):
      if (data[i] == str(0) and key[i] == str(0)) or (data[i] == str(1) and key[i] == str(1)):
        encrypted_message[i] = 0
      elif (data[i] == str(1) and key[i] == str(0)) or (data[i] == str(0) and key[i] == str(1)):
        encrypted_message[i] = 1

    return ''.join(map(str, encrypted_message))



def decryption(message,key):
    # function which recreates the original message using the key created between Alice and Bob
    # The message argument here is the encrypted message Bob recieves from Alice
    # This function is the same as the encryption function, using simple binary addition
    # Used by Bob
    return encryption(message,key)

# Convertir la lista en un string
def bin_to_num(lista):
  return int(''.join(map(str, lista)), 2)




In [112]:
#original

def string_converter(binary_lst):
    # this function creates an alphabet message from a the binary code
    word_length = 5 # as explained, every word is given as a 5 letter binary code for the message sent
    lista = []
    for element in binary_lst:
      lista.append(element)
    # split the list to different words
    temp = [lista[i:i + 5] for i in range(0, len(lista), 5)]

    bin_data = ''

    for i in range(len(temp)):
        temp[i] = [0,1,0] + temp[i] # add the uppercase digits, can change to lowercase using [0,1,1]
        str_temp = ' ' # notice the whitespace at the beginning of every letter
        res_temp = ' ' # another temporary variable
        for j in range(len(temp[i])):
            # loop through every word
            str_temp = str(temp[i][j]) # change to string values
            res_temp += str_temp
        bin_data += res_temp

    bin_data = bin_data[1:] # delete the first whitespace from the string

    binary_values = bin_data.split() # split on whitespace to convert each letter separatley

    res_string = ""

    for binary_value in binary_values:
        temp_int = int(binary_value, 2) # create binary value of item
        temp_char = chr(temp_int) # find the letter using the binary alphabet
        res_string += temp_char

    return res_string

def binary_converter(string):
    # converts string or message to binary list which corresponds to the message Alice wants to send
    # initialize lists
    temp_lst = []
    res_lst = []

    for character in string:
        # convert to binary
        temp_lst.append(bin(ord(character))[2:].zfill(8))

    for i in range(len(temp_lst)):
        # delete three first binary numbers as explained for message
        temp_lst[i] = temp_lst[i][3:]

    for j in range(len(temp_lst)):
        # create message as one list of binary numbers to send to Bob via the channel
        for k in range(len(temp_lst[j])):
            res_lst.append(int(temp_lst[j][k]))
    return res_lst


In [113]:
#nuevo (aun hay errores con los ejemplos de word )

def binary_converter(string):
    # Inicializamos una lista vacía para almacenar los valores binarios
    binary_list = []

    # Iteramos a través de cada carácter en la cadena
    for char in string:
        # Convertimos el carácter a su valor entero ASCII y luego a su representación binaria,
        # y lo agregamos a la lista
        binary_list.append(format(ord(char), '08b'))

    # Unimos los valores binarios en la lista y los devolvemos como una sola cadena
    return ''.join(binary_list)


def string_converter(binary):
    # Inicializamos una cadena vacía para almacenar el resultado
    result = ""

    # Iteramos a través de la cadena binaria en pasos de 8 caracteres
    for i in range(0, len(binary), 8):
        # Extraemos el bloque de 8 caracteres binarios
        binary_block = binary[i:i + 8]

        # Convertimos el bloque binario a su valor decimal y luego a su carácter ASCII correspondiente
        char = chr(int(binary_block, 2))

        # Agregamos el carácter al resultado
        result += char

    return result


# EXAMPLES

## Number: like a NIP

In [118]:
password = 12345

message = int_to_bin(password, 14)

key = create_key_bb84()
key = shorten_key(message, key)


encrypted_message = encryption(message, key)
decrypted_message = decryption(encrypted_message, key)
received_message = bin_to_num(decrypted_message)

print(f"sent message: {password}, messagge in binary format: {message}")
print(f"key: {key}, len: {len(key)}; message: {message}, len: {len(message)}")
print(f"encrypted message: {encrypted_message}")
print(f"decrypted message: {decrypted_message}")
print(f"received message: {received_message}")

sent message: 12345, messagge in binary format: 11000000111001
key: 11001110000001, len: 14; message: 11000000111001, len: 14
encrypted message: 00001110111000
decrypted message: 11000000111001
received message: 12345


## Word: like a PASSWORD




In [119]:
#text
original_message = "Google"

message = binary_converter(original_message)
message = ''.join(map(str, message))
key = create_key_bb84()
key = shorten_key(message, key)

encrypted_message = encryption(message, key)
decrypted_message = decryption(encrypted_message, key)
received_message = string_converter(decrypted_message)

print(f"sent message: {original_message}, messagge in binary format: {message}")
print(f"key: {key}, len: {len(key)}; message: {message}, len: {len(message)}")
print(f"encrypted message: {encrypted_message}")
print(f"decrypted message: {decrypted_message}")
print(f"received message: {received_message}")

sent message: Google, messagge in binary format: 010001110110111101101111011001110110110001100101
key: 010100101000110101001010001101010010100011010100, len: 48; message: 010001110110111101101111011001110110110001100101, len: 48
encrypted message: 000101011110001000100101010100100100010010110001
decrypted message: 010001110110111101101111011001110110110001100101
received message: Google


In [116]:
#text
original_message = "HswrTY@K"

message = binary_converter(original_message)
message = ''.join(map(str, message))
key = create_key_bb84()
key = shorten_key(message, key)


encrypted_message = encryption(message, key)
decrypted_message = decryption(encrypted_message, key)
received_message = string_converter(decrypted_message)

print(f"sent message: {original_message}, messagge in binary format: {message}")
print(f"key: {key}, len: {len(key)}; message: {message}, len: {len(message)}")
print(f"encrypted message: {encrypted_message}")
print(f"decrypted message: {decrypted_message}")
print(f"received message: {received_message}")

sent message: HswrTY@K, messagge in binary format: 0100100001110011011101110111001001010100010110010100000001001011
key: 1111011000001010001111011000001010001111011000001010001111011000, len: 64; message: 0100100001110011011101110111001001010100010110010100000001001011, len: 64
encrypted message: 1011111001111001010010101111000011011011001110011110001110010011
decrypted message: 0100100001110011011101110111001001010100010110010100000001001011
received message: HswrTY@K


## Message: like a Whatsapp message

In [120]:
original_message = "I am a future quantum leader (:"
print(f"original message: {original_message}")


message = str_to_binary(original_message)
message = ''.join(map(str, message))

print("---------------------------------------------")

print(f"binary message: {message}, len: {len(message)}")

key = create_key_bb84()

print(f"original key: {key} with {n_qubits} qubits, len: {len(key)}")

key = shorten_key(message, key)
print(f"adjusted key: {key}, len: {len(key)}")



encrypted_message = encryption(message, key)
decrypted_message = decryption(encrypted_message, key)
received_message = binary_to_str(decrypted_message)


print("---------------------------------------------")
print(f"encrypted message: {encrypted_message}")
print(f"decrypted message: {decrypted_message}")
print("---------------------------------------------")
print(f"received message: {received_message}")


original message: I am a future quantum leader (:
---------------------------------------------
binary message: 01001001001000000110000101101101001000000110000100100000011001100111010101110100011101010111001001100101001000000111000101110101011000010110111001110100011101010110110100100000011011000110010101100001011001000110010101110010001000000010100000111010, len: 248
original key: 11011010101101001110 with 28 qubits, len: 20
adjusted key: 11011010101101001110110110101011010011101101101010110100111011011010101101001110110110101011010011101101101010110100111011011010101101001110110110101011010011101101101010110100111011011010101101001110110110101011010011101101101010110100111011011010, len: 248
---------------------------------------------
encrypted message: 1001001110010100100011001100011001101110101110111001010010001011110111100011101010101111110001101000100010001011001111111010111111010101100000111101111100111011101101111001010010000001110011100010111110111110110100011001111110001011